In [29]:
import numpy as np
import pandas as pd

# Исходный код


In [30]:
__author__ = 'Chris Potts'


class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        The temperature parameter. Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        spk = self.speaker().T
        return rownorm(spk * self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Examples"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses']
    states = ['r1', 'r2']
    lex = pd.DataFrame([
        [0.0, 1.0],
        [1.0, 1.0]], index=msgs, columns=states)

# Задание 3
Референциальная игра:

In [41]:
    msgs = ['hat', 'glasses', 'mustache']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, 1.0, 0.0]], index=msgs, columns=states)

In [42]:
    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[1/3, 1/3, 1/3], costs=[0.0, 0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,r3,costs
hat,1.000000,1,0,0
glasses,0.000000,0,1,0
mustache,0.000000,1,0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Literal listener


,r1,r2,r3
hat,0.5,0.5,0.0
glasses,0.0,0.0,1.0
mustache,0.0,1.0,0.0



Pragmatic speaker


,hat,glasses,mustache
r1,1.000000,0.0,0.000000
r2,0.333333,0.0,0.666667
r3,0.000000,1.0,0.000000



Pragmatic listener


,r1,r2,r3
hat,0.75,0.25,0.0
glasses,0.00,0.00,1.0
mustache,0.00,1.00,0.0


## **Вопрос 1**
*Приведите скалярную импликатуру, которую можно вывести из таблицы прагматического слушающего* 

*Ответ*: Если говорящий говорит "усы", то имеется в виду точно второй человек, т.к. больше их ни у кого нет. Если говорящий говорит "очки", то точно третий, по той же самой причине. Если же он говорит "шляпа", то вероятность не стопроцентная, но наиболее вероятным является первый человек; если бы существовал более информативный признак, то говорящий назвал бы его. Это и есть скалярная импликатура

## **Вопрос 2**
*Как поменяется импликатура, если поменять исходные вероятности на P(r1) = 0.1, P(r2) = P(r3) = 0.45? Приведите соответствующую таблицу распределения вероятностей для прагматического слушающего и прокомментируйте её* 

*Ответ*:

In [34]:
    print("="*70 + "\nUNeven priors and all-0 message costs\n")
    prior_mod = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0.0, 0.0, 0.0])

    display_reference_game(prior_mod)

    print("\nLiteral listener")
    display(prior_mod.literal_listener())

    print("\nPragmatic speaker")
    display(prior_mod.speaker())

    print("\nPragmatic listener")
    display(prior_mod.listener())

UNeven priors and all-0 message costs



,r1,r2,r3,costs
hat,1.0,1,0,0
glasses,0.0,0,1,0
mustache,0.0,1,0,0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Literal listener


,r1,r2,r3
hat,0.181818,0.818182,0.0
glasses,0.000000,0.000000,1.0
mustache,0.000000,1.000000,0.0



Pragmatic speaker


,hat,glasses,mustache
r1,1.00,0.0,0.00
r2,0.45,0.0,0.55
r3,0.00,1.0,0.00



Pragmatic listener


,r1,r2,r3
hat,0.330579,0.669421,0.0
glasses,0.000000,0.000000,1.0
mustache,0.000000,1.000000,0.0


*Комментарий:* Единственное изменение, которое мы наблюдаем, это изменение вероятности для шляпы. Произошло это потому, что теперь, несмотря на то, что признак шляпы всё ещё самый информативный для первого человека, у него теперь меньше вероятность быть упомянутым (в 4.5 меньше , чем у второго), и слушающий об этом знает

# Задание 4
## **Вопрос 1**
*Приведите свой пример референциальной игры с четырьмя объектами (состояниями, r1, r2, r3, r4) и четырьмя высказываниями (m1, m2, m3, m4), в которой при равных исходных вероятностях объектов, alpha = 1, цене всех высказываний равной 0 вычисляется одна или две скалярные импликатуры. Покажите, что импликатуры вычисляются (таблица для прагматического слушающего + комментарий, описывающий импликатуру).*

In [43]:
    msgs = ['hat', 'glasses', 'mustache', 'scar']
    states = ['r1', 'r2', 'r3', 'r4']
    lex = pd.DataFrame([
        [1.0, 1.0, 1.0, 0.0],
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 1.0, 0.0, 1.0],
        [1.0, 0.0, 0.0, 1.0]], index=msgs, columns=states)

In [44]:
def display_game(mod):
  
    display_reference_game(mod)

    print("\nPragmatic listener")
    display(mod.listener())  

In [38]:
basic_mod_4 = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0])

display_reference_game(basic_mod_4)
print("\nLiteral listener")
display(basic_mod_4.literal_listener())

print("\nPragmatic speaker")
display(basic_mod_4.speaker())

print("\nPragmatic listener")
display(basic_mod_4.listener())

,r1,r2,r3,r4,costs
hat,1.00,1,1,0,0
glasses,0.00,0,1,0,0
mustache,0.00,1,0,1,0
scar,1.00,0,0,1,0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Literal listener


,r1,r2,r3,r4
hat,0.333333,0.333333,0.333333,0.0
glasses,0.000000,0.000000,1.000000,0.0
mustache,0.000000,0.500000,0.000000,0.5
scar,0.500000,0.000000,0.000000,0.5



Pragmatic speaker


,hat,glasses,mustache,scar
r1,0.40,0.00,0.0,0.6
r2,0.40,0.00,0.6,0.0
r3,0.25,0.75,0.0,0.0
r4,0.00,0.00,0.5,0.5



Pragmatic listener


,r1,r2,r3,r4
hat,0.380952,0.380952,0.238095,0.000000
glasses,0.000000,0.000000,1.000000,0.000000
mustache,0.000000,0.545455,0.000000,0.454545
scar,0.545455,0.000000,0.000000,0.454545


*Комментарий*: С очками всё понятно, потому что их больше никто не носит. Шрам считается самым информативным признаком для r1, усы - для r2; и то, и другое - по той причине, что все остальные признаки описывают большее количество человек (либо являются ложными для данного человека, что являлось бы нарушением максимы качества). С четвёртым получается интересно, так как, что бы мы ни сказали, на него не указывает ни один из признаков.

## **Вопрос 2**
Как меняются вероятности для прагматического слушающего с изменением параметра alpha (оптимальность, прагматичность, рациональность говорящего), параметра цены высказывания, с изменением исходных (априорных) вероятностей объектов? При каких значениях параметров импликатура (или импликатуры, если в Вашей игре их несколько) исчезает?

In [39]:
alpha4_mod_4 = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0], alpha=4)
alpha4_mod_2 = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0], alpha=2)
alpha4_mod_10 = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0], alpha=10)

display_game(alpha4_mod_4)
display_game(alpha4_mod_2)
display_game(alpha4_mod_10)

,r1,r2,r3,r4,costs
hat,1.00,1,1,0,0
glasses,0.00,0,1,0,0
mustache,0.00,1,0,1,0
scar,1.00,0,0,1,0
prior,0.25,0.25,0.25,0.25,
alpha,4.00,,,,



Pragmatic listener


,r1,r2,r3,r4
hat,0.482176,0.482176,0.035649,0.000000
glasses,0.000000,0.000000,1.000000,0.000000
mustache,0.000000,0.625483,0.000000,0.374517
scar,0.625483,0.000000,0.000000,0.374517


,r1,r2,r3,r4,costs
hat,1.00,1,1,0,0
glasses,0.00,0,1,0,0
mustache,0.00,1,0,1,0
scar,1.00,0,0,1,0
prior,0.25,0.25,0.25,0.25,
alpha,2.00,,,,



Pragmatic listener


,r1,r2,r3,r4
hat,0.430108,0.430108,0.139785,0.000000
glasses,0.000000,0.000000,1.000000,0.000000
mustache,0.000000,0.580645,0.000000,0.419355
scar,0.580645,0.000000,0.000000,0.419355


,r1,r2,r3,r4,costs
hat,1.00,1,1,0,0
glasses,0.00,0,1,0,0
mustache,0.00,1,0,1,0
scar,1.00,0,0,1,0
prior,0.25,0.25,0.25,0.25,
alpha,10.00,,,,



Pragmatic listener


,r1,r2,r3,r4
hat,0.499752,0.499752,0.000496,0.000000
glasses,0.000000,0.000000,1.000000,0.000000
mustache,0.000000,0.662835,0.000000,0.337165
scar,0.662835,0.000000,0.000000,0.337165


*Комментарий*: Чем выше альфа, тем сильнее вероятности "стягиваются" к одному, самому большому значению (или к нескольким большим, если их несколько одинаковых). Это объясняется тем, что чем прагматичнее говорящий, тем больше вероятность, что он применит наиболее информативное объяснение, и слушающий это знает 

In [46]:
prior4_mod_4 = RSA(lexicon=lex, prior=[0.4, 0.2, 0.2, 0.2], costs=[0.0, 0.0, 0.0, 0.0])
prior4_mod_2 = RSA(lexicon=lex, prior=[0.4, 0.1, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0])
prior4_mod_10 = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0])

display_game(prior4_mod_10)
display_game(prior4_mod_4)
display_game(prior4_mod_2)

,r1,r2,r3,r4,costs
hat,1.00,1,1,0,0
glasses,0.00,0,1,0,0
mustache,0.00,1,0,1,0
scar,1.00,0,0,1,0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Pragmatic listener


,r1,r2,r3,r4
hat,0.380952,0.380952,0.238095,0.000000
glasses,0.000000,0.000000,1.000000,0.000000
mustache,0.000000,0.545455,0.000000,0.454545
scar,0.545455,0.000000,0.000000,0.454545


,r1,r2,r3,r4,costs
hat,1.0,1,1,0,0
glasses,0.0,0,1,0,0
mustache,0.0,1,0,1,0
scar,1.0,0,0,1,0
prior,0.4,0.2,0.2,0.2,
alpha,1.0,,,,



Pragmatic listener


,r1,r2,r3,r4
hat,0.616438,0.239726,0.143836,0.000000
glasses,0.000000,0.000000,1.000000,0.000000
mustache,0.000000,0.526316,0.000000,0.473684
scar,0.740741,0.000000,0.000000,0.259259


,r1,r2,r3,r4,costs
hat,1.0,1,1,0,0
glasses,0.0,0,1,0,0
mustache,0.0,1,0,1,0
scar,1.0,0,0,1,0
prior,0.4,0.1,0.25,0.25,
alpha,1.0,,,,



Pragmatic listener


,r1,r2,r3,r4
hat,0.663188,0.113623,0.223188,0.000000
glasses,0.000000,0.000000,1.000000,0.000000
mustache,0.000000,0.295567,0.000000,0.704433
scar,0.710059,0.000000,0.000000,0.289941


Здесь можно заметить, что вероятности также "стягиваются" в зависимости от того, кто имеет большую вероятность быть упомянутым. 

In [49]:
cost4_mod_4 = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[0.0, 0.0, 0.0, 0.0])
cost4_mod_2 = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[6.0, 0.0, 0.0, 0.0])
cost4_mod_10 = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[6.0, 0.0, 6.0, 0.0])
cost4_mod_12 = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[-6.0, 0.0, 0.0, 0.0])
cost4_mod_20 = RSA(lexicon=lex, prior=[0.25, 0.25, 0.25, 0.25], costs=[-6.0, 0.0, -6.0, 0.0])

display_game(cost4_mod_4)
display_game(cost4_mod_2)
display_game(cost4_mod_10)
display_game(cost4_mod_12)
display_game(cost4_mod_20)

,r1,r2,r3,r4,costs
hat,1.00,1,1,0,0
glasses,0.00,0,1,0,0
mustache,0.00,1,0,1,0
scar,1.00,0,0,1,0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Pragmatic listener


,r1,r2,r3,r4
hat,0.380952,0.380952,0.238095,0.000000
glasses,0.000000,0.000000,1.000000,0.000000
mustache,0.000000,0.545455,0.000000,0.454545
scar,0.545455,0.000000,0.000000,0.454545


,r1,r2,r3,r4,costs
hat,1.00,1,1,0,6
glasses,0.00,0,1,0,0
mustache,0.00,1,0,1,0
scar,1.00,0,0,1,0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Pragmatic listener


...


Pragmatic listener


,r1,r2,r3,r4
hat,0.399934,0.399934,0.200132,0.0000
glasses,0.000000,0.000000,1.000000,0.0000
mustache,0.000000,0.666300,0.000000,0.3337
scar,0.666300,0.000000,0.000000,0.3337


,r1,r2,r3,r4,costs
hat,1.00,1,1,0,-6
glasses,0.00,0,1,0,0
mustache,0.00,1,0,1,-6
scar,1.00,0,0,1,0
prior,0.25,0.25,0.25,0.25,
alpha,1.00,,,,



Pragmatic listener


,r1,r2,r3,r4
hat,0.004099,0.993850,0.002051,0.000000
glasses,0.000000,0.000000,1.000000,0.000000
mustache,0.000000,0.995896,0.000000,0.004104
scar,0.500206,0.000000,0.000000,0.499794
